In [1]:
from transformers import AutoProcessor, AutoModelForZeroShotImageClassification
from PIL import Image
import torch

2025-04-18 12:28:16.601401: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744993696.956701  126174 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744993697.036065  126174 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744993697.870154  126174 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744993697.870194  126174 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744993697.870196  126174 computation_placer.cc:177] computation placer alr

In [2]:
# Load processor and model
processor = AutoProcessor.from_pretrained("patrickjohncyh/fashion-clip")
model = AutoModelForZeroShotImageClassification.from_pretrained("patrickjohncyh/fashion-clip")

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [3]:
# Load and prepare the image
image_path = "sweater.png"
image = Image.open(image_path).convert("RGB")

In [4]:
# Define fashion labels
candidate_labels = [
    "streetwear", "vintage style", "90s aesthetic", "oversized outfit",
    "y2k fashion", "boho", "classy evening wear", "minimalist", "grunge", "sweater"
]

# Preprocess input
inputs = processor(images=image, text=candidate_labels, return_tensors="pt", padding=True)

In [5]:
# Forward pass
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits_per_image  # shape: [1, num_labels]
    probs = logits.softmax(dim=1)

# Print ranked results
for label, score in sorted(zip(candidate_labels, probs[0]), key=lambda x: -x[1]):
    print(f"{label}: {score.item():.4f}")

sweater: 0.9970
boho: 0.0021
vintage style: 0.0004
minimalist: 0.0002
grunge: 0.0001
oversized outfit: 0.0001
streetwear: 0.0001
y2k fashion: 0.0000
90s aesthetic: 0.0000
classy evening wear: 0.0000


In [6]:
# BLIP-2 example using transformers
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch

In [7]:
# Load BLIP-2
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [8]:
# Load your image
image = Image.open("jeans.png").convert("RGB")

# Generate caption
inputs = processor(image, return_tensors="pt")
out = model.generate(**inputs)
caption = processor.decode(out[0], skip_special_tokens=True)
print("BLIP-2 Caption:", caption)

BLIP-2 Caption: a woman in a white top and blue pants
